## Advent of Code

Day 11: Reactor.

In [ ]:
from icecream import ic
import imageio.v2 as imageio
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import io
import os


def render(G: nx.Graph):
    pos = nx.circular_layout(G)

    nx.draw(G, pos,
            with_labels=True,
            font_size=10,
            node_color='lightgray',
            edge_color='lightgray',
            width=2
           )

    plt.show()


def render_and_save_paths(G: nx.Graph, source="you", target="out") -> list:
    filenames = []
    output_dir = "screenshots"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Setup the fixed layout.
    pos = nx.circular_layout(G)
    all_paths = list(nx.all_simple_paths(G, source=source, target=target))

    # Create the "Template" Figure.
    fig, ax = plt.subplots(figsize=(8, 6))

    # Draw the background (invisible nodes/edges to set the scale/bounds).
    nx.draw(G, pos, ax=ax, with_labels=False, node_color='white',
            edge_color='white', width=1, alpha=0.0)

    # Save the base state to a memory buffer using pickle.
    buf = io.BytesIO()
    pickle.dump(fig, buf)

    print(f"Found {len(all_paths)} paths. Saving each one individually...")

    for i, path in enumerate(all_paths):
        # RESTORE the figure from the buffer.
        buf.seek(0)
        current_fig = pickle.load(buf)
        current_ax = current_fig.gca() # Get current axes from restored figure.

        highlighted_edges = list(nx.utils.pairwise(path))
        highlighted_nodes = set(path)

        # Draw the specific path onto the restored axes.
        nx.draw_networkx_edges(
            G, pos, ax=current_ax,
            edgelist=highlighted_edges,
            edge_color='grey',
            width=3
        )

        nx.draw_networkx_nodes(
            G, pos, ax=current_ax,
            nodelist=list(highlighted_nodes),
            node_color='grey',
            node_size=500
        )

        # Draw source and target with special colors.
        nx.draw_networkx_nodes(G, pos, ax=current_ax, nodelist=[source],
                               node_color='green', node_size=500)
        nx.draw_networkx_nodes(G, pos, ax=current_ax, nodelist=[target],
                               node_color='red', node_size=500)

        # Add labels.
        nx.draw_networkx_labels(G, pos, ax=current_ax, font_size=10,
                                labels={node: node for node in highlighted_nodes})

        # Save the snapshot.
        filename = os.path.join(output_dir, f"output{i+1:03d}.png")
        current_fig.savefig(filename, bbox_inches='tight', dpi=50)
        filenames.append(filename)

        # Close the restored figure to free memory.
        plt.close(current_fig)

    # Close the original template figure.
    plt.close(fig)
    return filenames

In [ ]:
# Part 1.

file = open("input.txt","r")
contents = file.read()

G = nx.DiGraph()

for line in contents.split("\n"):
    from_vertex, other = line.split(": ")

    for to_vertex in other.split(" "):
        G.add_edge(from_vertex, to_vertex)

filenames = render_and_save_paths(G)

paths = list(nx.all_simple_paths(G, source="you", target="out"))

number_of_paths = len(paths)
ic(number_of_paths)

In [ ]:
print(filenames)

In [ ]:
# Make the screenshot images into an animated GIF.

with imageio.get_writer('day11.gif', mode='I', fps=10, loop=0) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

In [ ]:
# Part 2.

from icecream import ic
import networkx as nx

def count_paths_in_dag(G, source, target, memo=None):
    if memo is None:
        memo = {}

    if source == target:
        return 1
    if source in memo:
        return memo[source]

    count = 0
    for neighbor in G.neighbors(source):
        count += count_paths_in_dag(G, neighbor, target, memo)

    memo[source] = count
    return count


file = open("input.txt","r")
contents = file.read()

G = nx.DiGraph()

for line in contents.split("\n"):
    from_vertex, other = line.split(": ")

    for to_vertex in other.split(" "):
        G.add_edge(from_vertex, to_vertex)

svr_to_fft = count_paths_in_dag(G, "svr", "fft")
ic(svr_to_fft)

svr_to_dac = count_paths_in_dag(G, "svr", "dac")
ic(svr_to_dac)

fft_to_dac = count_paths_in_dag(G, "fft", "dac")
ic(fft_to_dac)

dac_to_fft = count_paths_in_dag(G, "dac", "fft")
ic(dac_to_fft)

fft_to_out = count_paths_in_dag(G, "fft", "out")
ic(fft_to_out)

dac_to_out = count_paths_in_dag(G, "dac", "out")
ic(dac_to_out)

total = svr_to_fft * fft_to_dac * dac_to_out + svr_to_dac * dac_to_fft * fft_to_out
ic(total)